In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch


In [ ]:
# Загрузка токенизатора и модели RuGPT3
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model = AutoModelForSeq2SeqLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [ ]:
# Подготовка данных для fine-tuning
train_texts = ["Входной текст 1", "Входной текст 2", ...]
train_summaries = ["Суммаризация 1", "Суммаризация 2", ...]

# Токенизация и кодирование данных
train_encodings = tokenizer(train_texts, train_summaries, truncation=True, padding=True)

# Преобразование данных в формат PyTorch тензоров
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings["input_ids"]),
    torch.tensor(train_encodings["attention_mask"]),
    torch.tensor(train_encodings["labels"])
)

In [ ]:
# Функция для вычисления метрик (например, BLEU) во время обучения (опционально)
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # Декодирование и преобразование тензоров в список строк
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Вычисление BLEU
    bleu_score = sacrebleu.corpus_bleu(pred_str, [label_str]).score

    return {"bleu": bleu_score}

In [ ]:
# Настройка тренировочного цикла
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    logging_dir="./logs",
)

In [ ]:
# Создание объекта Trainer для fine-tuning
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# Запуск fine-tuning
trainer.train()